# Création du Dataset en scrappant les infos depuis une liste d'url

## Imports

In [6]:
import scraping_info_perfume as sip
import pandas as pd
import numpy as np
import time

Fragrantica_dataset = pd.read_csv("Ajout_Fragrantica_dataset.csv")
urls_a_scraper = pd.read_csv("list_url_add_on.csv")
urls_deja_scrapees = Fragrantica_dataset['url'].tolist()
#Choix de la lettre à scraper
#lettre = ('A')
#df_a_scraper = df[df['Marque'].str.startswith(lettre)]
# Exclure les lignes déjà présentes dans Fragrantica_dataset
df_a_scraper = urls_a_scraper[~urls_a_scraper['url'].isin(urls_deja_scrapees)] 
df_a_scraper.describe()

,brand,name,url
count,2,2,2
unique,2,2,2
top,Escentric Molecules,Molecule 01 Limited Edition 15 Years,https://www.fragrantica.com/perfume/Escentric-...
freq,1,1,1


## Test pour voir le temps d'attente optimal à scraper 50 parfums

In [3]:
import scraping_info_perfume as sip
import pandas as pd

batch_size = 49
'''
# Charger le dataset existant
Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
url_deja_scrape = set(Fragrantica_dataset['url'].tolist())

# Filtrer les URLs déjà scrapées
df_a_scraper = df_a_scraper[~df_a_scraper['Lien du Parfum'].isin(url_deja_scrape)]
'''
df_a_scraper = pd.read_csv('liste_url_mal_scrapee.csv')
# Initialiser les DataFrames pour les données scrapées
scraped_data_1 = pd.DataFrame()
scraped_data_2 = pd.DataFrame()

# Fonction pour scraper un batch et mesurer le temps d'exécution
def scrape_batch(batch, batch_num):
    liste_url = batch["url"].tolist()
    start_time = time.time()
    scraped_data = sip.scraping_multi_perfume_info(liste_url)
    end_time = time.time()
    execution_time = end_time - start_time
    scraped_data.to_csv(f"scraped_data_{batch_num}.csv", index=False)
    return scraped_data, execution_time

# Scraper le premier batch
batch1 = df_a_scraper.iloc[0 : batch_size]
scraped_data_1, execution_time_1 = scrape_batch(batch1, 1)
print(f"Temps d'exécution pour batch 1 : {execution_time_1:.1f} secondes.")
print(f"Soit {execution_time_1 / batch_size:.1f} secondes par URL.")

# Attendre avant de scraper le deuxième batch
time.sleep(1200)

# Scraper le deuxième batch
batch2 = df_a_scraper.iloc[batch_size : 2 * batch_size]
scraped_data_2, execution_time_2 = scrape_batch(batch2, 2)
print(f"Temps d'exécution pour batch 2 : {execution_time_2:.1f} secondes.")
print(f"Soit {execution_time_2 / batch_size:.1f} secondes par URL.")




Temps d'exécution pour batch 1 : 349.2 secondes.
Soit 7.1 secondes par URL.
Temps d'exécution pour batch 2 : 0.1 secondes.
Soit 0.0 secondes par URL.


In [4]:

scraped_data1 = pd.read_csv(f"scraped_data_1.csv")
scraped_data_1.to_csv("Ajout_Fragrantica_dataset.csv", index=False)
#afficher le nomre de ligne de chaque dataframe
# Concaténer les deux DataFrames
#combined_data = pd.concat([Fragrantica_dataset, scraped_data1])
#combined_data.drop_duplicates(subset=['url'], inplace=True)
#combined_data.to_csv("Ajout_Fragrantica_dataset.csv", index=False)

'''
scraped_data2 = pd.read_csv(f"scraped_data_2.csv")
combined_data = pd.concat([Fragrantica_dataset, scraped_data2])
combined_data.drop_duplicates(subset=['url'], inplace=True)
combined_data.to_csv("Fragrantica_dataset.csv", index=False)


Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
print(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")
print(f"✅ Les données ont été ajoutées à Fragrantica_dataset.csv sans doublons.")
'''



'\nscraped_data2 = pd.read_csv(f"scraped_data_2.csv")\ncombined_data = pd.concat([Fragrantica_dataset, scraped_data2])\ncombined_data.drop_duplicates(subset=[\'url\'], inplace=True)\ncombined_data.to_csv("Fragrantica_dataset.csv", index=False)\n\n\nFragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")\nprint(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")\nprint(f"✅ Les données ont été ajoutées à Fragrantica_dataset.csv sans doublons.")\n'

## Le script qui scrape les infos par batch de 49 lignes

In [ ]:
batch_size = 10
num_batches = len(df_a_scraper) // batch_size + (1 if len(df_a_scraper) % batch_size != 0 else 0)
print(f"📦 TAILLE DES BATCHS = {batch_size}")

#1) Obtenir les URLs déjà scrapées
print(f"📂 Nous avons déja {len(urls_deja_scrapees)} URLs dans le dataset Fragrantica.")
print("--------------------------------------------------------------------------------")

time.sleep(1200)

num_url_scrap = 0
#2) Itérer sur les batches 
for i in range(num_batches):
    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')	
    print(f"📡 Traitement du batch {i+1}/{num_batches}...")

    batch = df_a_scraper.iloc[i * batch_size : (i + 1) * batch_size]
    liste_url = batch["url"].tolist()

    try:
        start_time = time.time()
        scraped_data = sip.scraping_multi_perfume_info(liste_url)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"⏱️ Temps d'exécution du scraping sur le batch{i+1} : {execution_time:.1f} secondes.")
        print(f"⏱️ Soit un total de {execution_time/len(liste_url):.1f} secondes par URL.")
        
        # Filtrer les lignes valides
        scraped_data = scraped_data.dropna(subset=['nom_parfum'])

        file_name = f"scrap_batch_{i+1}.csv"
        scraped_data.to_csv(file_name, index=False) # Sauvegarde du batch sans index inutile
        print(f"✅ Batch {i+1} enregistré dans {file_name} avec les lignes valides.")
        num_url_scrap += len(scraped_data)

    except Exception as e:
        print(f"⚠️ Erreur lors du scraping du batch {i+1}: {e}")




print(f"\n🎉 AU FINAL {num_url_scrap} URLs ont été scrapées avec succès !")




📦 TAILLE DES BATCHS = 10
📂 Nous avons déja 2699 URLs dans le dataset Fragrantica.
--------------------------------------------------------------------------------


## Code pour ajouter nos nouvelles lignes à 'Fragrantica_dataset.csv' 

In [5]:
num_premier_batch = 1
num_dernier_batch = 39
for i in range(num_premier_batch-1, num_dernier_batch):
    scraped_data = pd.read_csv(f"scrap_batch_{i+1}.csv")
    Fragrantica_dataset = pd.read_csv("Ajout_Fragrantica_dataset.csv")

    # Concaténer les deux DataFrames
    combined_data = pd.concat([Fragrantica_dataset, scraped_data])

    # Supprimer les doublons
    combined_data.drop_duplicates(subset=['url'], inplace=True)

    # Sauvegarder le dataset combiné
    combined_data.to_csv("Ajout_Fragrantica_dataset.csv", index=False)
    print(f"✅ Les données du batch {i+1} ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.")


✅ Les données du batch 1 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 2 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 3 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 4 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 5 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 6 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 7 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 8 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 9 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 10 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 11 ont été ajoutées à Ajout_Fragrantica_dataset.csv sans doublons.
✅ Les données du ba